# ** Interactive Baseline Correction Plot **
<div style="margin-top:10px; text-align:justify";>This function uses Bokeh to create an interactive visualization of the original and baseline-corrected LIBS spectrum for a single replicate.

<br>💡 What the code does:</br>

- Applies baseline correction to one sample column using the AsLS algorithm (pybaselines)

- Creates an interactive plot with:

- Hover tooltips for precise values

- Zooming, panning, and saving tools

- A legend you can click to toggle visibility

- Lets you focus on a custom wavelength range (optional)

📌 Why it’s useful:

- Visualizes how baseline correction improves your spectra

- Easy to explore the before and after correction effects

- Perfect for presentations or exploring problem regions
</div>

In [1]:
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource
from pybaselines import whittaker as pl

def plot_corrected_interactive(df, sample_col, wavelength_col='wavelength', zoom_range=None):
    
    x = df[wavelength_col]
    y = df[sample_col]

    # Apply baseline correction using AsLS
    baseline, _ = pl.asls(y)
    corrected = (y - baseline).clip(lower=0)

    # Apply zoom range if specified
    if zoom_range:
        mask = (x >= zoom_range[0]) & (x <= zoom_range[1])
        x = x[mask]
        y = y[mask]
        corrected = corrected[mask]

    # Prepare Bokeh data source
    source = ColumnDataSource(data={
        'x': x,
        'original': y,
        'corrected': corrected
    })

    # Create interactive plot
    p = figure(
        title=f"LIBS Spectrum – {sample_col} (Interactive)",
        x_axis_label="Wavelength (nm)",
        y_axis_label="Intensity (a.u.)",
        width=1000,
        height=400,
        tools="pan,wheel_zoom,box_zoom,reset,save,hover",
        tooltips=[
            ("Wavelength", "@x{0.0}"),
            ("Original", "@original{0.0}"),
            ("Corrected", "@corrected{0.0}")
        ]
    )

    p.line('x', 'original', source=source, legend_label="Original", line_width=2, color="blue", alpha=0.6)
    p.line('x', 'corrected', source=source, legend_label="Corrected", line_width=2, color="green", alpha=0.8)

    p.legend.click_policy = "hide"
    p.legend.location = "top_left"
    p.add_layout(p.legend[0], 'right')

    show(p)

In [ ]:
# Select one replicate from a sample
sample_col = get_sample_columns(SAMPLE_Raw_df, 'BAM-308')[0]

# Plot with your defined region of interest (471–520 nm)
plot_corrected_interactive(SAMPLE_Raw_df, sample_col, zoom_range=(200, 1000))